<a href="https://colab.research.google.com/github/Debottam/MachinLearningEx/blob/master/customizedModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TensorFlow Customized models:

In [2]:
import tensorflow as tf
import tensorflow.keras as keras

print("tensorflow: ", tf.__version__)
print("keras: ", keras.__version__) 

tensorflow:  2.3.0
keras:  2.4.0


Some tensorflow layers are without any weight
keras.layers.Flatten
keras.layers.ReLU

In [ ]:
# lambda layer
exponential_layer = keras.layers.Lambda(lambda x: tf.exp(x))
exponential_layer

layers are customized from `keras.layers.Layer`

In [ ]:
class MyDense(keras.layers.Layer):
  def __init__ (self, units, activation = None, **kwargs):
    super().__init__(**kwargs)
    self.units = units
    self.activation = keras.activations.get(activation)
  
  def build(self, batch_input_shape):
    self.kernel = self.add_weight(name = 'kernel', shape = [batch_input_shape[-1], self.units], initializer = "glorot_normal")
    self.bias = self.add_weight(name = "bias", shape = [self.units], initializer = 'zeros')
    super().build(batch_input_shape)

  def call(self, X):
    return self.activation(X @ self.kernel + self.bias)

  def compute_output_shape(self, batch_input_shape):
    return tf.TensorShape(batch_input_shape.as_list()[:-1]+ [self.units])
  
  def get_config(self):
    base_config = super().get_config()
    return {**base_config, 'units':self.units, 'activation': keras.activations.serialize(self.activation)}

MultiInput custom layers

In [ ]:
class MyMultiLayer(keras.layers.Layer):
  def call (self, X):
    X1, X2 = X1
    return [X1+X2, X1*X2, X1/X2]

  def compute_output_shape(self, batch_input_shape):
    b1, b2 = batch_input_shape
    return [b1, b1, b1]

Layer to add gaussian noise during training

In [ ]:
class myGaussianNoise(keras.layers.Layer):
  def __init__(self, stdev, **kwargs):
    super().__init__(**kwargs)
    self.stdev = stdev
  
  def call(self, X, training = None):
    if training:
      noise = tf.random.normal(tf.shape(X), stdev = self.stdev)
      return X + noise 
    else:
      return X
  
  def compute_output_shape (self, batch_input_shape):
    return batch_input_shape


Customizing Models

In [5]:
class ResidualBlock(keras.layers.Layer):
  def __init__(self, n_layers, n_neurons, **kwargs):
    super().__init__(**kwargs)
    self.hidden = [keras.layers.Dense(n_neurons, activation = "elu", kernel_initializer = "he_normal") for _ in range(n_layers)]
  
  def call (self, inputs):
    Z = inputs
    for layer in self.hidden:
      Z = layer(Z)
    return inputs+Z

class ResidualRegressor(keras.Model):
  def __init__(self, output_dim, **kwargs):
    super().__init__(**kwargs)
    self.hidden1 = keras.layers.Dense(30, activation = 'elu', kernel_initializer = "he_normal")
    self.block1 = ResidualBlock(2, 30)
    self.block2 = ResidualBlock(2, 30)
    self.out = keras.layers.Dense(output_dim)
  
  def call(self, inputs):
    Z = self.hidden1(inputs)
    for _ in range(1+3):
      Z = self.block1(Z)
    Z = self.block2(Z)
    return self.out(Z)

Customized model with loss with 5 hidden layers

In [6]:
class ReconstructingRegressor(keras.Model):
  def __init__(self, output_dim, **kwargs):
    super().__init__(**kwargs)
    self.hidden = [keras.layers.Dense(30, activation = "selu", kernel_initializer="lecun_normal") for _ in range(5)]
    self.out = keras.layers.Dense(output_dim)
  
  def build(self, batch_input_shape):
    n_inputs = batch_input_shape[-1]
    self.reconstruct = keras.layers.Dense(n_inputs)
    super().build(batch_input_shape)

  def call(self, inputs):
    Z = inputs
    for layer in self.hidden:
      Z = layer(Z)
    reconstruction = self.reconstruct(Z)
    recon_loss = tf.reduce_mean(tf.square(inputs - reconstruction))
    self.add_loss(0.05*recon_loss)
    return self.out(Z)

